<a href="https://colab.research.google.com/github/shizoda/education/blob/main/agent/LangGraph_and_Gradio(2)_MultiAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. マルチエージェントシステムとは何か
大規模言語モデル (LLM) は高性能ですが、単体のモデルでは以下のような課題があります。

1.  **自己修正の難しさ** : 自身が生成した誤りを、同じ思考プロセスの中で発見し修正することが困難です。
2.  **役割の固定化** : 創造的なアイデア出しと厳密な論理検証など、相反する思考モードを同時に維持するのが苦手です。
3.  **複雑なタスクの処理** : 長期的で複雑な工程を一気に処理させると、指示の追従性が低下します。

 **マルチエージェントシステム** は、複数の LLM (または異なる役割を与えられた同一の LLM) を連携させることで、これらの課題を解決するアプローチです。作成者とレビューアのように役割を分担させ、対話を通じて出力をブラッシュアップさせることで、単一のエージェントでは到達できない品質の回答を得ることが可能になります。

ここではLangGraph を用いて 出題者 (Teacher) と 解答・評価者 (Student) という2つのエージェントが対話し、高品質な教材を作成するシステムを構築します。

In [ ]:
# 必要なライブラリのインストール
!pip install -qU langgraph langchain-openai langchain-community gradio==6.2.0 termcolor

# 2. 環境設定とモデルの準備
OpenRouter 経由で `nvidia/nemotron-3-nano-30b-a3b` を使用します。

In [ ]:
import os, sys
from google.colab import userdata
from termcolor import colored
from langchain_openai import ChatOpenAI

# APIキーの読み込み
try:
    os.environ["OPENROUTER_API_KEY"] = userdata.get("OPENROUTER_API_KEY")
    print(colored("OPENROUTER_API_KEY loaded.", "green"))
except Exception as e:
    print(colored("Error: OPENROUTER_API_KEY not found in Secrets.", "red"))

# モデル名とプロバイダの定義
MODEL_NAME = "nvidia/nemotron-3-nano-30b-a3b"

llm = ChatOpenAI(
    model=MODEL_NAME,
    openai_api_base="https://openrouter.ai/api/v1",
    api_key=os.environ["OPENROUTER_API_KEY"],
    streaming=True,
    model_kwargs={
        "extra_body": {
            "provider": {
                "order": ["deepinfra"],
                "allow_fallbacks": False
            }
        }
    }
)

# 3. ステート (状態) の定義
マルチエージェントシステムにおいて最も重要なのが、エージェント間で共有される 状態 (State) の設計です。
LangGraph では、この State がエージェント間をリレー形式で渡されていきます。

ここでは、以下の情報を管理します。
* `messages`: 全エージェントの会話履歴。これを見ることで、相手が何を言ったかを把握します。
* `topic`: 人間が与えたお題 (例：中学2年の一次関数)。
* `current_turn`: 現在の往復回数。無限ループを防ぐために使用します。
* `max_turns`: 会話を終了する上限回数。

In [ ]:
from typing import Annotated, Sequence, TypedDict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langgraph.graph import add_messages

class MultiAgentState(TypedDict):
    # add_messages は、新しいメッセージをリストに追加 (append) する挙動を定義します
    messages: Annotated[Sequence[BaseMessage], add_messages]
    topic: str
    current_turn: int
    max_turns: int

# 4. エージェント (ノード) の定義
各エージェントは、システムプロンプトによって異なる役割 (ペルソナ) を与えられます。

### Teacher エージェント (出題者)
* **役割** : 日本の学習指導要領に基づいた、良質な問題を作成する。
* **動作** : 会話履歴を確認し、Student からのフィードバックがあれば問題を修正する。なければ新規作成する。

### Student エージェント (解答・評価者)
* **役割** : 出された問題を解き、その品質を評価する。
* **動作** : 問題を解いてみせ、さらに難易度は適切か、曖昧さはないかを指摘する。

In [ ]:
# 共通の制約プロンプト
COMMON_PROMPT = """
【出力ルール】
1. 数式は必ず $ または $$ で囲まれた LaTeX 形式で記述してください。
2. 強調のための太字 **...** を使用する場合、その前後に必ず半角スペースを入れてください（例： **重要** ）。
3. 出力は 50 行以内としてください。
4. 比喩表現を避け、落ち着いたトーンで客観的な解説を行ってください。「！」や大袈裟な表現は使用しないでください。
"""

# Teacher (先生) ノード
def teacher_node(state: MultiAgentState):
    topic = state["topic"]
    messages = state["messages"]

    if not messages:
        content = f"テーマ {topic} について、日本の学習指導要領に沿った入試レベルの問題を 1 つ作成してください。"
    else:
        last_feedback = messages[-1].content
        content = f"生徒から以下のフィードバックがありました：\n\n{last_feedback}\n\nこれを受けて、元の問題をより良く修正・改善してください。"

    # 共通ルールを結合
    full_content = content + "\n" + COMMON_PROMPT

    print(colored(f"\n\n【Teacher】", "cyan", attrs=["bold"]))

    # HumanMessage として渡し、LLM に返信を促す
    response = llm.invoke([HumanMessage(content=full_content)])
    response.name = "Teacher"

    return {
        "messages": [response],
        "current_turn": state["current_turn"] + 1
    }

# Student (生徒) ノード
def student_node(state: MultiAgentState):
    messages = state["messages"]

    # Teacher が出した最新の問題を取得
    last_problem = messages[-1].content

    content = f"""
    あなたは少し出来の悪い生徒です。先生が作成した以下の問題を解き、その品質をレビューしてください。

    【問題】
    {last_problem}

    【手順】
    1. まず、その問題を解いて解答を示してください。
    2. 次に、その問題が読みやすいか、難易度は適切か、入試問題として適切かを評価してください。
    3. 改善点があれば具体的に提案してください。正直な気持ちとして書いてください。
    """

    # 共通ルールを結合
    full_content = content + "\n" + COMMON_PROMPT

    print(colored(f"\n\n【Student】", "yellow", attrs=["bold"]))

    # 先生の回答を入力として受け取り、それに対する返答を生成させる
    response = llm.invoke([HumanMessage(content=full_content)])
    response.name = "Student"

    return {"messages": [response]}

# 5. グラフの構築と制御フロー
定義したノードを連結し、ワークフローを構築します。
ここでは `should_continue` という条件付きエッジ (Conditional Edge) を定義し、指定したターン数に達するまで Teacher ⇔ Student のループを繰り返すように設計します。

これにより、 **作成 → レビュー → 修正 → 再レビュー** という品質向上のサイクルが自動的に回ります。

In [ ]:
from langgraph.graph import StateGraph, START, END
from IPython.display import Image, display

# 終了判定関数
def should_continue(state: MultiAgentState):
    if state["current_turn"] >= state["max_turns"]:
        return END
    return "student"

# グラフの構築
workflow = StateGraph(MultiAgentState)

# ノード登録
workflow.add_node("teacher", teacher_node)
workflow.add_node("student", student_node)

# エッジ登録
workflow.add_edge(START, "teacher")

# Teacher の後に終了判定を入れる
workflow.add_conditional_edges(
    "teacher",
    should_continue,
    {
        "student": "student",
        END: END
    }
)

# Student からは Teacher に戻して修正させる
workflow.add_edge("student", "teacher")

# コンパイル
app = workflow.compile()

# グラフ構造を表示

try:
    display(Image(app.get_graph().draw_mermaid_png()))
except Exception:
    print("Graph visualization error: Ensure dependencies are installed.")

# 6. 実行と動作確認
まずコンソール上で動作を確認します。
お題として 連立方程式の利用（速さと時間） を与え、2ターンの会話 (作成→指摘→修正→指摘) を行わせます。

In [ ]:
import sys
from IPython.display import Markdown, display, clear_output

initial_state = {
    "topic": "中学2年 数学（連立方程式の文章題）",
    "messages": [],
    "current_turn": 0,
    "max_turns": 2
}

print(colored(f"Target Topic: {initial_state['topic']}", "green", attrs=["bold"]))
print("-" * 60)

# 実行ループ
# stream_mode="updates" は、1つのノード（Teacher または Student）が完了するたびにイベントを返します
for event in app.stream(initial_state, stream_mode="updates"):

    # event には {'ノード名': {'messages': [...], ...}} という形式でデータが入っています
    node_name = list(event.keys())[0]
    last_message = event[node_name]["messages"][-1]

    # 役割に応じたラベルと色の設定
    sender_name = last_message.name if hasattr(last_message, "name") else node_name.capitalize()

    # Markdown 表示用の文字列を作成
    # 数式が含まれる場合があるため、適切な余白を持ってレンダリングします
    markdown_output = f"## 👤 {sender_name}\n\n{last_message.content}\n\n---"

    # 即座に Markdown を表示
    display(Markdown(markdown_output))

    # ユーザーに入力を求める（ここで一時停止）
    # display の直後に置くことで、表示を確認してから入力を受け付けます
    user_choice = input(f"\n[{sender_name} の応答が完了] 続けますか？ (y/n): ")

    if user_choice.lower() != 'y':
        print(colored("\nユーザーによってプロセスが中断されました。", "red", attrs=["bold"]))
        break

print(f"\n\n{colored(' --- すべてのプロセスが完了しました --- ', 'green', attrs=['bold'])}")

# 7. Human-in-the-loop (人間介入型) 実行

LangGraph の **`interrupt_after`** 機能を使用し、AI が発言するたびに処理を一時停止させます。

**【操作方法】**
1. AI が発言した後、入力欄が表示されます。
2. **そのまま進める場合**: 何も入力せずに `Enter` を押してください。
3. **指示を出す場合**: 「もっと難しくして」「具体的な数字を使って」などと入力すると、その指示が **HumanMessage** として会話に割り込みます。
4. **終了する場合**: `q` を入力してください。

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage
import uuid

# --- 1. ノード関数の再定義（アドバイス対応） ---

def teacher_node_hitl(state: MultiAgentState):
    topic = state["topic"]
    messages = state["messages"]

    if not messages:
        content = f"テーマ {topic} について、入試レベルの問題を作成してください。"
    else:
        # Student のフィードバックを受けて修正
        last_msg = messages[-1]
        content = f"生徒からフィードバックがありました：\n「{last_msg.content}」\n\nこれを受けて問題を修正してください。"

    response = llm.invoke([HumanMessage(content=content + "\n" + COMMON_PROMPT)])
    response.name = "Teacher"
    return {"messages": [response], "current_turn": state["current_turn"] + 1}

def student_node_hitl(state: MultiAgentState):
    messages = state["messages"]

    # Teacher の最新の問題を取得
    teacher_msgs = [m for m in messages if hasattr(m, "name") and m.name == "Teacher"]
    last_problem = teacher_msgs[-1].content if teacher_msgs else "（問題が見つかりません）"

    # ユーザーからのアドバイスがあるか確認
    advice_content = ""
    if isinstance(messages[-1], HumanMessage):
        advice_content = f"""
        【ユーザーからのアドバイス】
        「{messages[-1].content}」

        ※ 上記はあくまで外部からのアドバイスです。
        参考にするかどうかは自分で判断し、自分の言葉で解答・評価を行ってください。
        無理にアドバイスに従う必要はありません。
        """

    content = f"""
    以下の問題を解き、品質を評価してください。

    【問題】
    {last_problem}

    {advice_content}
    """

    response = llm.invoke([HumanMessage(content=content + "\n" + COMMON_PROMPT)])
    response.name = "Student"
    return {"messages": [response]}


# --- 2. グラフの再構築 ---

memory = MemorySaver()
workflow_hitl = StateGraph(MultiAgentState)

workflow_hitl.add_node("teacher", teacher_node_hitl)
workflow_hitl.add_node("student", student_node_hitl)

workflow_hitl.add_edge(START, "teacher")
workflow_hitl.add_conditional_edges(
    "teacher",
    should_continue,
    {"student": "student", END: END}
)
workflow_hitl.add_edge("student", "teacher")

app_hitl = workflow_hitl.compile(
    checkpointer=memory,
    interrupt_after=["teacher"]
)

from IPython.display import Image, display

print(colored("--- ワークフローの可視化 ---", "cyan", attrs=["bold"]))
print("以下の図は、Teacher（出題）と Student（評価）の循環構造を示しています。")
print("今回の設定では、 'student' ノードの処理が終わった直後にシステムが一時停止し、")
print("あなたの指示（Teacherへの修正命令）を待つ仕組みになっています。\n")

try:
    # Mermaid 形式でグラフを描画して表示
    display(Image(app_hitl.get_graph().draw_mermaid_png()))
except Exception as e:
    print(colored(f"グラフの描画に失敗しました: {e}", "red"))
    print("※ 依存ライブラリの関係で表示できない場合がありますが、動作には影響ありません。")

In [ ]:
# --- 3. 対話ループの実行 ---

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

initial_state = {
    "topic": "中学2年 数学（連立方程式の文章題）",
    "messages": [],
    "current_turn": 0,
    "max_turns": 6
}

print(colored(f"Target Topic: {initial_state['topic']}", "green", attrs=["bold"]))
print(colored("--- Human-in-the-loop Session Start (Advice Mode) ---", "green"))
print("Teacher(出題)の後でのみ停止します。Student(生徒)へアドバイスを送れます。\n")

events = app_hitl.stream(initial_state, config=config, stream_mode="updates")

while True:
    # 1. AIのターンを実行
    try:
        for event in events:
            node_name = list(event.keys())[0]
            node_state = event[node_name]

            # タプル対策
            if isinstance(node_state, tuple):
                if len(node_state) == 0: continue
                node_state = node_state[0]

            if "messages" in node_state and node_state["messages"]:
                msg = node_state["messages"][-1]
                sender = msg.name if hasattr(msg, "name") else node_name.capitalize()

                # Markdown表示
                display(Markdown(f"## 👤 {sender}\n\n{msg.content}\n\n---"))

    except Exception as e:
        print(f"Loop Error: {e}")
        pass

    # 2. 終了判定
    snapshot = app_hitl.get_state(config)
    if not snapshot.next:
        print(colored("\n--- 対話終了 ---", "green", attrs=["bold"]))
        break

    # 3. 停止中の処理 (Teacherの後でのみここに来る)
    print(colored("\n[一時停止中] Studentへのアドバイスを入力してください:", "grey"))
    user_input = input("アドバイス (Enterでスキップ, qで終了): ")

    if user_input.lower() == 'q':
        break

    if user_input:
        print(colored(f" >> アドバイスを送りました: 「{user_input}」", "blue"))
        # ユーザー入力を履歴に追加
        app_hitl.update_state(config, {"messages": [HumanMessage(content=user_input)]})

    print(colored("\n(AIが思考中...)", "grey"))
    events = app_hitl.stream(None, config=config, stream_mode="updates")

# 演習課題: 役割の変更
現在定義されている `teacher_node` (出題者) と `student_node` (解答者) のプロンプトを変更し、
**料理のレシピを考案するシェフ** と **栄養バランスをチェックする栄養士** の対話システムに変更してください。